# Główny kod
Dokumentacja API [MFW](https://www.imf.org/external/datamapper/api/help)

Słownik pańsw UE

In [ ]:
eu_country_codes = {
    'AUT': 'Austria','BEL': 'Belgium', 'BGR': 'Bulgaria', 'HRV': 'Croatia',
    'CYP': 'Cyprus', 'CZE': 'Czech Republic', 'DNK': 'Denmark', 'EST': 'Estonia',
    'FIN': 'Finland', 'FRA': 'France', 'DEU': 'Germany', 'GRC': 'Greece',
    'HUN': 'Hungary','IRL': 'Ireland', 'ITA': 'Italy', 'LVA': 'Latvia',
    'LTU': 'Lithuania', 'LUX': 'Luxembourg', 'MLT': 'Malta', 'NLD': 'Netherlands',
    'POL': 'Poland', 'PRT': 'Portugal', 'ROU': 'Romania', 'SVK': 'Slovakia',
    'SVN': 'Slovenia', 'ESP': 'Spain', 'SWE': 'Sweden' }

Głowna funkcja obsługująca pobieranie:

In [ ]:
import requests, json
import pandas as pd

def get_data(variable: str, country_codes: dict, periods: range):
  MAIN_URL = "https://www.imf.org/external/datamapper/api/v1/"

  # Selection countries
  countries = ""
  for i in country_codes.keys():
    countries = countries + "/" + i

  # Selection periods
  periods_str = ""
  for i in periods:
    periods_str = periods_str + str(i) + ","
  periods_str = periods_str[:-1]

  # URL
  URL =  MAIN_URL + variable + countries + "?periods=" + periods_str
  print(URL)

  # Download the contents of the frame page
  response = requests.get(URL)

  # This command transforms the text into a structure
  if response.status_code == 200:
    data_json = json.loads(response.text)

  data_df = pd.DataFrame(data_json['values'][variable])
  return data_df

# Wykres 1: Skumulowany wzrost PKB w prognozie MFW
Ściągamy tempo wzrostu PKB:

In [ ]:
df_gdp = get_data("NGDP_RPCH", eu_country_codes, range(2025,2030))
df_gdp

Skumulowana wartośc - suma wszystkich elementów kolumny:

In [ ]:
df_cum_gpd = df_gdp.sum()
df_cum_gpd = df_cum_gpd.sort_values(ascending=False)
df_cum_gpd.to_csv("df_cum_gpd.csv")

# Wykres 2: PKB per capita
Na początek drobna tabela do użycia w trakcie pisania tekstu - udział w światowym PKB:

In [ ]:
# CEE
visegrad_country_codes = {
    'CZE': 'Czech Republic',
    'HUN': 'Hungary',
    'POL': 'Poland',
    'SVK': 'Slovakia',
}

df_share_world_ppl_CEE = get_data("PPPSH", visegrad_country_codes, range(2025,2030))
df_share_world_ppl_CEE

In [ ]:
weimar_country_codes = {
    'FRA': 'France',
    'DEU': 'Germany',
    'POL': 'Poland'
}

df_share_world_ppl_weimar = get_data("PPPSH", weimar_country_codes, range(2025,2030))
df_share_world_ppl_weimar

Teraz przeanalizujemy PKB per capita PPP oraz mierzonego w dolarach:

In [22]:
# USD
df_gdp_pc_usd = get_data("NGDPDPC", eu_country_codes, range(2025,2030))

# PPS
df_gdp_pc_ppp = get_data("PPPPC", eu_country_codes, range(2025,2030))

https://www.imf.org/external/datamapper/api/v1/NGDPDPC/AUT/BEL/BGR/HRV/CYP/CZE/DNK/EST/FIN/FRA/DEU/GRC/HUN/IRL/ITA/LVA/LTU/LUX/MLT/NLD/POL/PRT/ROU/SVK/SVN/ESP/SWE?periods=2025,2026,2027,2028,2029
https://www.imf.org/external/datamapper/api/v1/PPPPC/AUT/BEL/BGR/HRV/CYP/CZE/DNK/EST/FIN/FRA/DEU/GRC/HUN/IRL/ITA/LVA/LTU/LUX/MLT/NLD/POL/PRT/ROU/SVK/SVN/ESP/SWE?periods=2025,2026,2027,2028,2029


Przeliczenie jako odsetka do Niemiec:

In [24]:
# USD - % Niemiec
df_gdp_pc_vs_DEU_USD = df_gdp_pc_usd.div(df_gdp_pc_usd['DEU'], axis=0)
df_gdp_pc_vs_DEU_USD.to_csv("df_gdp_pc_vs_DEU_USD.csv")

# PPP - % Niemiec
df_gdp_pc_vs_DEU_PPP = df_gdp_pc_ppp.div(df_gdp_pc_ppp['DEU'], axis=0)
df_gdp_pc_vs_DEU_PPP.to_csv("df_gdp_pc_vs_DEU_PPP.csv")

Na koniec PKB Polski to jaki odsetek kolejnych krajów:

In [30]:
df_gdp_pc_PLN_to_others_USD = df_gdp_pc_usd.apply(lambda x: df_gdp_pc_usd['POL'] / x)
df_gdp_pc_PLN_to_others_USD[["CZE", "DEU", "FRA", "ESP", "ITA"]].to_csv("df_gdp_pc_PLN_to_others_USD.csv")

In [31]:
df_gdp_pc_PLN_to_others_PPP = df_gdp_pc_ppp.apply(lambda x: df_gdp_pc_ppp['POL'] / x)
df_gdp_pc_PLN_to_others_PPP[["CZE", "DEU", "FRA", "ESP", "ITA"]].to_csv("df_gdp_pc_PLN_to_others_PPP.csv")